In [1]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans

In [2]:
#Data Preparation
#user_feature table
user_feat = pd.DataFrame(np.random.randint(low=0, high=10, size=(100, 3)),columns=['Age','Sex','Location'])


user_feat['Age'] = pd.DataFrame(np.random.randint(5,100, size = (100,1)))
user_feat['Location'] = np.random.choice(['J&K', 'Haryana', 'Punjab', 'U.P.', 'Maharashtra', 'South India'],100)
user_feat['Sex']= np.random.choice(["M","F"],100)

num = np.arange(500,5000)
num = num[0:len(user_feat)]
user_feat['user_guid'] = num
print type(user_feat)

#spice_feature table
spice_feat = pd.DataFrame(np.random.randint(low=0, high=2, size=(100, 6)),columns=['spice1','spice2','spice3','spice4','count','Location'])
num = np.arange(60000,70000)
num = num[0:len(spice_feat)]

spice_feat['food_id'] = num
spice_feat['count'] = np.random.randint(0,5, size=len(spice_feat))
spice_feat['Location'] = np.random.choice(['J&K', 'Haryana', 'Punjab', 'U.P.', 'Maharashtra', 'South India'],100)

l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}

spice_feat  = spice_feat.replace({'Location':l_dict})

#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']


<class 'pandas.core.frame.DataFrame'>


In [3]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'J&K':2, 'Haryana':5, 'Punjab':6, 'U.P.':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})

In [4]:
print "enter the user id"

input_user_id = input()



enter the user id
555


In [5]:
kmeans = KMeans(n_clusters = 10,random_state=0).fit(user_feat)
user_feat['label'] = kmeans.labels_


index = np.where(user_feat['user_guid']==input_user_id)[0]
print index
user_label = user_feat.ix[50,'label']
user_feat = user_feat.sort(['label','Location'],ascending=[1,0])
user = user_feat.drop_duplicates(subset=['Location','label'])
user = user.groupby('label')['Location'].nlargest(3)
loc = user[user_label].tolist()
print loc

[55]
[15, 10, 8]


C:\Users\ravindra\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [24]:

newspice_feat = spice_feat.set_index('food_id')
spice = newspice_feat[newspice_feat['Location'].isin(loc)]
s_spice =  spice.sort('count',ascending=[0])
s_index = food_user.index[food_user[input_user_id]==0]
new_spices = s_spice[s_spice.index.isin(s_index)]
print new_spices

         spice1  spice2  spice3  spice4  count  Location
food_id                                                 
60044         0       0       1       0      4         8
60012         0       1       0       1      4         8
60079         1       0       1       0      4        15
60072         1       0       1       1      4         8
60066         0       0       0       1      4         8
60059         1       1       1       1      4        15
60020         0       0       0       0      4        10
60093         0       1       1       1      4         8
60007         0       0       0       0      4        10
60045         1       1       0       1      3        10
60019         0       0       0       0      3        10
60022         0       0       0       0      3        15
60025         0       0       1       1      3        10
60068         1       1       1       1      3        15
60048         0       1       0       0      3        15
60042         1       0       1

C:\Users\ravindra\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [25]:
recommend = []
for i in range(min(3,len(new_spices))):
    recommend.append(new_spices.index[i])
    
if(len(recommend)==0):
    other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
    other_state_spice = other_state_spice.sort('count',ascending= [0])
    new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]

    for i in range(min(3,len(new_other_state))):
        recommend.append(new_other_state.index[i])

    
print recommend    

[60044, 60012, 60079]


In [35]:
spice = np.array(spice_feat.iloc[:,0:4])
user_df = pd.DataFrame(np.random.randint(low=-1, high=2, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)

user = np.array(user_df)
global spice
global user

In [36]:
# we have user_table which will give rating to a particular dish
# content based recommeder system
# we have 2 tables with us

#one is food - spices table and another is food-user rating table
#we will use tf-idf approach
#matrix of spice will contain only of 1 and 0's so need to normalize it

#normalization along the row
def spice_norm():
    
    num_attr = np.sum(spice,axis=1)
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0): 
            doc_vec.append(np.sqrt(spice[i,:]*1.0/num_attr[i]))
        else:
            doc_vec.append(spice[i,:])
    return np.array(doc_vec)      
    
    
    
def attr_norm():
    
    num_attr = np.sum(spice,axis=0)
    
    doc_vec = []
    for i in range(num_attr.shape[0]):
        if(num_attr[i] != 0):
            doc_vec.append(spice[:,i]*1.0 / num_attr[i])
        else:
            doc_vec.append(spice[:,i])
    return np.array(doc_vec)

def user_v(spice_mat):
    user_vec = []
    
    
    #print user[:,0]
    #print spice_mat[:,0]
    
    for i in range(user.shape[1]):
        user_vec.append([np.dot(user[:,i],spice_mat[:,j]) for j in range(spice_mat.shape[1])])
    
    print (user_vec)
    return np.array(user_vec)
    
def weight_predict(spice_mat):
        
    user_vec = user_v(spice_mat)
    
    preds = []
    #print spice_mat
    for i in range(spice_mat.shape[0]):
        preds.append([np.dot(spice_mat[i,:],user_vec[j,:]) for j in range(user_vec.shape[0])])

    return np.array(preds)
    
    
def idf_weight_pred():
    spice_mat = spice_norm()
    attr_wght = attr_norm()
    #print attr_wght
    weighted_spice = []
    
    for i in range(spice_mat.shape[0]):
        weighted_spice.append([np.dot(spice_mat[i,:],attr_wght[:,j]) for j in range(attr_wght.shape[0])])
        
    preds  = weight_predict(spice_mat = np.array(weighted_spice))     
    
    return np.array(preds)
    


preds = idf_weight_pred()

#print preds.shape

[[0.020206717183450074, -0.0075326694234750524, -0.01001236998614826, -0.037262658036557122], [0.16460709374593852, 0.078827200288625651, 0.1449876147056923, 0.028652344362066921], [0.049139678208066931, 0.1875596408402759, -0.0024073271757255807, 0.01040915714401637], [0.10924390061150928, 0.29513287670343819, 0.091508873692546677, 0.21250763259313238], [-0.027152034811404324, 0.0041822774198978195, -0.033720577306328145, -0.02826844068892622], [-0.051655925566256922, -0.1476757897703404, 0.0041946492314219874, -0.012208000613542554], [0.22351280952450783, 0.14359805847777551, 0.1240189796307446, -0.022675720316690011], [0.21944665015089862, 0.21113716850353448, 0.021212742704289467, -0.065197281357299075], [0.054311205778723518, 0.063737435282440635, 0.0029329237636765773, -0.055564407813738753], [0.20233586271942694, 0.22215853459834559, 0.16217442892927153, 0.11267290657421494], [0.25159439236901748, 0.22024781094385104, 0.1265728552578862, 0.05010931484602911], [0.0421905627542948

In [39]:
outputdf = pd.DataFrame(data = preds,index = newspice_feat.index ,columns = user_df.columns)

s_index = food_user.index[food_user[input_user_id]==0]
output = outputdf[outputdf.index.isin(s_index)]
print user_df.columns
newoutput = output[input_user_id].sort_values(ascending = False)
print newoutput
food_to_recommend = list(newoutput[0:3].index)

print food_to_recommend

Int64Index([584, 580, 582, 583, 585, 579, 581, 586, 587, 525, 520, 521, 522,
            529, 531, 524, 526, 530, 523, 527, 555, 556, 552, 547, 549, 548,
            553, 554, 557, 550, 551, 505, 510, 500, 507, 502, 504, 509, 501,
            506, 508, 503, 593, 597, 589, 591, 592, 596, 590, 594, 599, 598,
            588, 595, 565, 561, 567, 562, 564, 568, 559, 560, 563, 558, 566,
            543, 545, 546, 540, 541, 534, 536, 538, 542, 544, 515, 513, 514,
            519, 512, 516, 511, 518, 517, 570, 573, 569, 575, 572, 574, 571,
            576, 577, 578, 528, 533, 535, 539, 537, 532],
           dtype='int64', name=u'user_guid')
food_id
60059    0.018801
60068    0.018801
60069    0.018801
60001    0.017461
60073    0.017461
60093    0.017461
60036    0.017461
60040    0.017461
60055    0.017461
60063    0.017236
60080    0.017236
60031    0.015906
60018    0.015724
60088    0.015724
60045    0.015724
60072    0.014707
60091    0.014707
60095    0.014707
60012    0.014055
60014   